In [1]:
# Simple Repository Migration - No API Required

from pathlib import Path
import shutil
import subprocess
from datetime import datetime

class SimpleRepositoryMigrator:
    """Simple repository migration without AI features"""
    
    def __init__(self, repo_path="."):
        self.repo_path = Path(repo_path).resolve()
        self.backup_dir = self.repo_path / "backup_before_migration"
        
    def analyze_current_structure(self):
        """Show current repository structure"""
        current_files = []
        current_dirs = []
        
        for item in self.repo_path.iterdir():
            if item.name.startswith('.') or item.name == 'backup_before_migration':
                continue
                
            if item.is_file():
                current_files.append(item.name)
            elif item.is_dir():
                current_dirs.append(item.name)
        
        print("Current Repository Structure:")
        print(f"Files: {current_files}")
        print(f"Directories: {current_dirs}")
        
        return current_files, current_dirs
    
    def create_backup(self):
        """Create backup of current state"""
        self.backup_dir.mkdir(exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_path = self.backup_dir / f"backup_{timestamp}"
        backup_path.mkdir()
        
        for item in self.repo_path.iterdir():
            if item.name not in ['.git', 'backup_before_migration']:
                if item.is_file():
                    shutil.copy2(item, backup_path)
                elif item.is_dir():
                    shutil.copytree(item, backup_path / item.name)
        
        print(f"✅ Backup created at: {backup_path}")
        return backup_path
    
    def create_directory_structure(self):
        """Create basic directory structure"""
        dirs_to_create = [
            "projects/01-mcp-basics",
            "projects/02-github-integration",
            "projects/03-travel-assistant",
            "docs/learning-notes",
            "docs/progress",
            "docs/references",
            "experiments",
            "resources",
            "scripts"
        ]
        
        for dir_path in dirs_to_create:
            full_path = self.repo_path / dir_path
            full_path.mkdir(parents=True, exist_ok=True)
            
            # Create simple README
            readme_path = full_path / "README.md"
            if not readme_path.exists():
                with open(readme_path, "w") as f:
                    f.write(f"# {dir_path.split('/')[-1].replace('-', ' ').title()}\n\nCreated: {datetime.now().strftime('%Y-%m-%d')}\n")
    
    def move_file_with_git(self, source, target_dir):
        """Move file using git mv to preserve history"""
        try:
            # Ensure target directory exists
            target_dir.mkdir(parents=True, exist_ok=True)
            
            # Use git mv
            cmd = ['git', 'mv', str(source), str(target_dir / source.name)]
            subprocess.run(cmd, cwd=self.repo_path, check=True)
            print(f"Moved {source.name} to {target_dir}")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Warning: Could not move {source.name} with git: {e}")
            return False
    
    def migrate_files(self):
        """Migrate files based on simple patterns"""
        moves_made = []
        
        # Define file patterns and targets
        file_mappings = {
            'mcp-reference-notebook.md': 'projects/01-mcp-basics',
            'travel_assistant_mini.ipynb': 'projects/03-travel-assistant',
            'travel_assistant_complete.ipynb': 'projects/03-travel-assistant',
            'github_mcp.ipynb': 'projects/02-github-integration',
        }
        
        # Move specific files
        for filename, target_dir in file_mappings.items():
            file_path = self.repo_path / filename
            if file_path.exists():
                target = self.repo_path / target_dir
                if self.move_file_with_git(file_path, target):
                    moves_made.append((filename, target_dir))
        
        # Move any remaining notebooks to experiments
        for file_path in self.repo_path.glob('*.ipynb'):
            if file_path.is_file():
                target = self.repo_path / 'experiments'
                if self.move_file_with_git(file_path, target):
                    moves_made.append((file_path.name, 'experiments'))
        
        # Move any .py files to scripts
        for file_path in self.repo_path.glob('*.py'):
            if file_path.is_file():
                target = self.repo_path / 'scripts'
                if self.move_file_with_git(file_path, target):
                    moves_made.append((file_path.name, 'scripts'))
        
        return moves_made
    
    def create_simple_readme(self):
        """Create a simple updated README"""
        readme_content = """# Learn Git and MCP

This repository is organized into:

- `projects/` - Learning projects
- `docs/` - Documentation
- `experiments/` - Test notebooks
- `resources/` - Reference materials
- `scripts/` - Utility scripts

Migration completed on: {date}
"""
        readme_path = self.repo_path / "README.md"
        with open(readme_path, "w") as f:
            f.write(readme_content.format(date=datetime.now().strftime('%Y-%m-%d')))
    
    def commit_changes(self, moved_files):
        """Create a commit for the migration"""
        # Add all changes
        subprocess.run(['git', 'add', '.'], cwd=self.repo_path)
        
        # Create commit message
        commit_message = f"""Migrate to monorepo structure

Reorganized repository for better organization:
- Created projects/, docs/, experiments/ structure
- Moved files to appropriate directories
- Preserved file history

Files moved:
"""
        for file, target in moved_files:
            commit_message += f"- {file} -> {target}\n"
        
        try:
            subprocess.run(['git', 'commit', '-m', commit_message], cwd=self.repo_path, check=True)
            subprocess.run(['git', 'tag', '-a', 'monorepo-migration', '-m', 'Repository structure migration'], 
                         cwd=self.repo_path, check=True)
            print("\n✅ Migration committed and tagged!")
        except subprocess.CalledProcessError as e:
            print(f"Error creating commit: {e}")

# Run the migration
def run_simple_migration():
    """Run the simple migration process"""
    print("🚀 Simple Repository Migration")
    print("=" * 30)
    
    migrator = SimpleRepositoryMigrator()
    
    # Analyze current structure
    print("\n1. Current repository structure:")
    migrator.analyze_current_structure()
    
    # Confirm proceeding
    proceed = input("\nProceed with migration? (y/n): ").lower()
    if proceed != 'y':
        print("Migration cancelled.")
        return
    
    # Create backup
    print("\n2. Creating backup...")
    migrator.create_backup()
    
    # Create directory structure
    print("\n3. Creating new directory structure...")
    migrator.create_directory_structure()
    
    # Migrate files
    print("\n4. Moving files...")
    moved_files = migrator.migrate_files()
    
    # Update README
    print("\n5. Updating README...")
    migrator.create_simple_readme()
    
    # Commit changes
    print("\n6. Committing changes...")
    migrator.commit_changes(moved_files)
    
    print("\n✨ Migration complete!")
    print("\nNext steps:")
    print("1. Check the new structure")
    print("2. Push to remote: git push origin main --tags")

# Execute the migration
run_simple_migration()

🚀 Simple Repository Migration

1. Current repository structure:
Current Repository Structure:
Files: ['migrate-to-monorepo.ipynb_simpel.ipynb', 'README.md']
Directories: ['projects', 'learning_notes', 'mcp_fundamentals', 'git_examples']



Proceed with migration? (y/n):  y



2. Creating backup...
✅ Backup created at: /Users/kerstinforsberg/repos/learn_git_and_mcp/backup_before_migration/backup_20250505_151446

3. Creating new directory structure...

4. Moving files...

5. Updating README...

6. Committing changes...
[main 2323b57] Migrate to monorepo structure
 17 files changed, 1848 insertions(+), 41 deletions(-)
 create mode 100644 backup_before_migration/backup_20250505_151446/.gitignore
 create mode 100644 backup_before_migration/backup_20250505_151446/README.md
 create mode 100644 backup_before_migration/backup_20250505_151446/migrate-to-monorepo.ipynb_simpel.ipynb
 create mode 100644 backup_before_migration/backup_20250505_151446/projects/mcp-travel-assistant/mcp-reference-notebook.md
 create mode 100644 backup_before_migration/backup_20250505_151446/projects/mcp-travel-assistant/travel_assistant_complete.ipynb
 create mode 100644 backup_before_migration/backup_20250505_151446/projects/mcp-travel-assistant/travel_assistant_mini.ipynb
 create mode 10

fatal: not under version control, source=migrate-to-monorepo.ipynb_simpel.ipynb, destination=experiments/migrate-to-monorepo.ipynb_simpel.ipynb
